# Boosting

---
## Experimental Setup

### Libraries & Settings

In [1]:
import os              # General OS commands
import numpy as np     # NumPy
import pandas as pd    # Python Data Analysis Library
import zipfile         # Compress/decompress ZIP files
import sqlite3         # SQLite3 Database Driver
import re              # Regular Expressions
import datetime        # Date/Time functions

In [2]:
from sklearn import ensemble, metrics, tree

In [3]:
# Never truncate columns, display all the data
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', -1)

# Display floating-point numbers with 4 decimals in `pandas.DataFrame`
pd.options.display.float_format = '{:,.4f}'.format

import matplotlib.pyplot as plt
# Display MatPlotLib stuff inline
%matplotlib inline

### Database

In [4]:
zip_filename = "../../data/ee-insee-2015_custom-sqlite.zip"
eedb = zip_filename.replace("-sqlite.zip", ".sqlite")

if not os.path.exists(eedb):
    with zipfile.ZipFile(zip_filename) as zip_file:
        zip_file.extractall("../../data/")

In [5]:
with sqlite3.connect(eedb) as con:
    query = "SELECT * FROM eec15_custom"
    eec15 = pd.read_sql_query(query, con)

In [6]:
# Create a ("female" x "enfant") interaction variable
bool_ = eec15[["enfants_", "female_"]].astype(bool)
eec15["female_enfants_"] = (bool_.enfants_ & bool_.female_).astype(int)

# Drop data we don't need
eec15 = eec15[eec15.age60_ == 0]
eec15 = eec15.drop("age60_", 1)

---
## Boosting


In [7]:
# Build a list of parameters to include in the model, using regex
# https://www.datarobot.com/blog/multiple-regression-using-statsmodels/
filters = {
    "age": "^age[0-9]{2}_$",
    "diploma": "^dip[0-9]{2}_$",
    "etranger": "^etranger_$",
    "domtom": "^domtom_$",
#     "trim": "^trim$",
    "female": "^female_$",
    "enfants": "^enfants_$",
#     "female_enfants": "^female_enfants_$",
    "region": "^region[1-2]_$"
}
params = {k: sorted([x for x in eec15.columns if re.match(r, x)]) for (k, r) in filters.items()}

# Avoid the dummy variable trap
params = {k: (v if len(v) == 1 else v[:-1]) for (k, v) in params.items()}
params["region"] += ["region2_"]

In [8]:
trims = ["t{}".format(x) for x in sorted(eec15["trim"].unique())]
eec15_ = {t: eec15[eec15.trim == int(t[1])] for t in trims}
X = {t: eec15_[t][sum(params.values(), [])] for t in eec15_}
y = {t: eec15_[t]["actop_"] for t in eec15_}

### Train the model using `trim=1` data

In [9]:
simple_tree_sk = tree.DecisionTreeClassifier(criterion='entropy', max_depth=10,random_state=0).fit(X["t1"], y["t1"])
boost_sk = ensemble.AdaBoostClassifier(
    base_estimator=simple_tree_sk,
    n_estimators=100,
    learning_rate=0.1,
    random_state=0
).fit(X["t1"], y["t1"])

### Apply the model to test data (`trim=2/3/4`)

In [10]:
for t in trims:
#     predicted = random_forest_sk.predict_proba(X[t]).T[1]
#     predicted = np.where(predicted < 0.5, 0, 1)
    predicted = boost_sk.predict(X[t])
    print "Accuracy ({}): {}".format(t, metrics.accuracy_score(y[t], predicted))

# Note: the two methods for computing `predicted` are equivalent (THRESHOLD = 0.5)

Accuracy (t1): 0.777410143057
Accuracy (t2): 0.780638214565
Accuracy (t3): 0.772162940133
Accuracy (t4): 0.773814763805


In [11]:
for t in trims:
    predicted_proba = boost_sk.predict_proba(X[t]).T[1]
    print "ROC Area-Under-Curve ({}): {}".format(t, metrics.roc_auc_score(y[t], predicted_proba))

ROC Area-Under-Curve (t1): 0.820594780348
ROC Area-Under-Curve (t2): 0.818967354886
ROC Area-Under-Curve (t3): 0.809582476961
ROC Area-Under-Curve (t4): 0.812174478218


### Accuracy of prediction of `1` and `0`

In [12]:
for actop_ in [0, 1]:
    print "===== actop_ = {} =====".format(actop_)
    for t in trims:
        y_zero = y[t][y[t] == actop_]
        X_zero = X[t][X[t].index.map(lambda x: x in y_zero)]
    
        predicted = boost_sk.predict(X_zero)
        print "Accuracy ({}): {}".format(t, metrics.accuracy_score(y_zero, predicted))
    print

===== actop_ = 0 =====
Accuracy (t1): 0.916532497246
Accuracy (t2): 0.91491410111
Accuracy (t3): 0.912762233889
Accuracy (t4): 0.915493570636

===== actop_ = 1 =====
Accuracy (t1): 0.521221814777
Accuracy (t2): 0.527269791288
Accuracy (t3): 0.503742863185
Accuracy (t4): 0.507683504986



---
## Marginal Effects

###  By modifying the dataset ("passage de tout le monde en licence")

In [13]:
brute_force = pd.DataFrame()
for category in filters:
    for pivot in params[category]:
        non_pivots = [x for x in params[category] if x != pivot]
        
        X_one = X["t1"].copy() 
        X_one[non_pivots] = 0
        X_one[pivot] = 1
        
        # Note mean() on the other dimension as compared to simple tree
        # This makes no difference on the mean(), but gives us the right std()
        proba_one = boost_sk.predict_proba(X_one).T[1]
        proba_t1 = boost_sk.predict_proba(X["t1"]).T[1]
        
        brute_force[pivot] = proba_one - proba_t1    # these are now vectors

In [14]:
marginal_effects = pd.DataFrame()
marginal_effects["brute_force_mean"] = brute_force.mean()
marginal_effects["brute_force_std"] = brute_force.std()
marginal_effects.transpose()

,etranger_,age15_,age30_,age40_,dip10_,dip11_,dip30_,dip31_,dip33_,dip41_,dip42_,dip50_,dip60_,dip70_,female_,enfants_,region1_,region2_,domtom_
brute_force_mean,0.0134,0.0116,-0.0133,-0.0223,-0.0369,-0.0996,-0.0611,-0.0279,-0.1678,-0.0023,-0.0107,0.0015,0.0251,0.0395,0.0096,-0.0196,0.0032,-0.0232,-0.0067
brute_force_std,0.1633,0.0950,0.0844,0.0823,0.1063,0.1817,0.2027,0.1060,0.2280,0.1114,0.0809,0.0753,0.1015,0.1839,0.0535,0.0843,0.0510,0.1434,0.1411


---
## Mean Marginal Effects (*bootstrap*)

We use the *bootstrap* method to calculate a confidence interval for our marginal effects. The *bootstrap* method consists in successive **random samplings with replacement** of the original database.

**Note: this is computationally very intensive – be careful or else**

### Computation

In [15]:
n = 1                           # number of iterations
k = eec15.index.size              # number of samples per iteration

In [21]:
index = sum(params.values(), [])
marginal_effects_iterations = pd.DataFrame(index=index)

start = datetime.datetime.now()
print "start: {}".format(start)

for i in range(n):  
    # take a sample of the DataFrame
    eec15_sample = eec15.sample(k, replace=True)

    # separate parameters and output
    X_sample = eec15_sample[index]
    y_sample = eec15_sample["actop_"]

    # build the boost for the sample
    sample_boost_sk = ensemble.AdaBoostClassifier(
        base_estimator=simple_tree_sk,
        n_estimators=100,
        learning_rate=0.1,
        random_state=0
    ).fit(X_sample, y_sample)

    # calculate marginal effects for that sample
    brute_force = pd.Series()
    for category in filters:
        for pivot in params[category]:
            non_pivots = [x for x in params[category] if x != pivot]
        
            X_one = X_sample.copy() 
            X_one[non_pivots] = 0
            X_one[pivot] = 1
        
            proba_one = sample_boost_sk.predict_proba(X_one).T[1].mean()
            proba_base = sample_boost_sk.predict_proba(X_sample).T[1].mean()
            brute_force[pivot] = proba_one - proba_base
    
    # store the results in the dataframe
    marginal_effects_iterations[i] = brute_force
    
    # dump 50 just to be safe
    if not i % 50:
        marginal_effects_iterations.to_csv("csv/boosting_{}_{}.csv".format(n, i))

    # print completed iteration
    print i+1,

    with open("csv/boosting_log", "w") as f:
        f.write(str(i+1)+"\n")
    
print
end = datetime.datetime.now()
print "ran {} iterations in {}".format(n, end-start)

start: 2017-03-01 18:31:33.432421
1
ran 1 iterations in 0:03:40.498502


In [17]:
marginal_effects_iterations.to_csv("csv/boosting_{}.csv".format(n))

### Mean/Standard Deviation (across iterations)

In [18]:
marginal_effects_summary = pd.DataFrame()
marginal_effects_summary["mean"] = marginal_effects_iterations.mean(axis=1)
marginal_effects_summary["std"] = marginal_effects_iterations.std(axis=1)

marginal_effects_summary.transpose()

,etranger_,age15_,age30_,age40_,dip10_,dip11_,dip30_,dip31_,dip33_,dip41_,dip42_,dip50_,dip60_,dip70_,female_,enfants_,region1_,region2_,domtom_
mean,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
std,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### 95% Confidence Interval

In [19]:
interval_size = 0.99
alpha = (1 - interval_size)/2
ix = int(round(alpha*n))      # ix has to be an integer

interval = pd.DataFrame(index=["lower", "upper"])
for category in filters:
    for pivot in params[category]:
        me_pivot = marginal_effects_iterations.loc[pivot].sort_values()
        interval[pivot] = [me_pivot.iloc[ix], me_pivot.iloc[-ix]]

interval

IndexError: single positional indexer is out-of-bounds

---
## Most important parameters
_ranked by **entropy importance**_ (**TODO**: where did Bastien get this?!)

In [ ]:
entropy = pd.DataFrame(boost_sk.feature_importances_, index=X["t1"].columns, columns=["entropy"])
#entropy_sorted = entropy.sort_values(by="entropy", ascending=False).transpose()
entropy.transpose()

---
# Odds Ratios

The odds ratios formula is:
$$\frac{p/(1-p)}{q/(1-q)} = \frac{p(1-q)}{q(1-p)}$$

where $p$ and $q$ are the probability of being **unemployed**.

In [ ]:
odds = pd.Series()
for category in filters:
    for pivot in params[category]:
        non_pivots = [x for x in params[category] if x != pivot]
        
        X_one = X["t1"].copy() 
        X_one[non_pivots] = 0
        X_one[pivot] = 1

        proba_one = boost_sk.predict_proba(X_one).T[1].mean()
        odds[pivot] = proba_one/(1-proba_one)  

In [ ]:
odds_ratios = pd.DataFrame(index=odds.index, columns=odds.index)
for col in odds.index:
    for row in odds.index:
        odds_ratios[col][row] = odds[col]/odds[row]

In [ ]:
# odds_ratios     # col, row = odds[col]/odds[row]
odds_ratios.T   # row, col = odds[row]/odds[col]  

---
## Odds Ratios (2)
**TODO**: add the other Odds Ratios formula

In [ ]:
odds_ratios_2 = pd.DataFrame()
for category in filters:
    for pivot in params[category]:
        non_pivots = [x for x in params[category] if x != pivot]
        
        X_one = X["t1"].copy() 
        X_one[non_pivots] = 0
        X_one[pivot] = 1
        proba_one = boost_sk.predict_proba(X_one).T[1].mean()
        odds_one = proba_one/(1-proba_one)
        
        X_zero = X["t1"].copy()
        X_zero[non_pivots] = 0
        X_zero[pivot] = 0
        proba_zero = boost_sk.predict_proba(X_zero).T[1].mean()
        odds_zero = proba_zero/(1-proba_zero)
        
        odds_ratios_2.loc[pivot, "ratio"] = odds_one/odds_zero

odds_ratios_2.transpose()